<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://cdn.miami.edu/_assets-common/images/system/um-logo-gray-bg.png" alt="Miami Logo" style="height: 98px;">
</div>

<div style="float:right; width:98 px; height:98px;">
<img src="https://media.licdn.com/dms/image/C4E0BAQFlOZSAJABP4w/company-logo_200_200/0/1548285168598?e=2147483647&v=beta&t=g4jl8rEhB7HLJuNZhU6OkJWHW4cul_y9Kj_aoD7p0_Y" alt="STI Logo" style="height: 98px;">
</div>


<h1>Compute Harmonics on the sFWRD Database</h1>
By: Tyler M. Fenske
    <br>
Last Edited: 2024-02-01
<br>
<br>    
<br>
This notebook applies harmonics analysis to various forecast and renalysis data as part of the data pre-processing for future forecasting work. 
<br>    
<br>
Note: some of the datasets were too large to be processed all at once. The work around is to use Split_BigData.ipynb that takes a given model and splits each yearly variable file into 4 distinct quadrant files spatially and rejoining them after harmonic processing. 
<br>    
<br>
<div style="clear:both"></div>
</div>

<hr style="height:2px;">

### Imports & Functions

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import os
import glob

In [3]:
%run File_concat_mod_functions.ipynb
#include many of the existing functions to handle the NOAA S2S database

In [4]:
def model_concat(model_list, model):

    '''
    Takes a list of files and concatenates them along the time dimension.

    Inputs:
    
    model_list: (list of str) list of filenames to be opened and concatenates them along the time dimension 
    model: (str) one of the six available model outputs in the database (does not work for UFS)
    
    Outputs:

    df1: (xarray dataset) combined dataset of all files along the time dimension

    '''

    

    if model == 'CONUS404':
        cc_dim = 'Time'
        df1 = xr.open_dataset(model_list[0]).astype('float32') #.chunk(get_chunk_database(model))
    
        for f in model_list[1:]:
            df2 = xr.open_dataset(f).astype('float32') #.chunk(get_chunk_database(model))
            df1 = xr.concat([df1, df2], dim = cc_dim)
    else:
        cc_dim = 'time'
    
        df1 = xr.open_dataset(model_list[0]).astype('float32') #.chunk(get_chunk_database(model))
    
        for f in model_list[1:]:
            df2 = xr.open_dataset(f).astype('float32')#.chunk(get_chunk_database(model))
            print(f)
            df1 = xr.concat([df1, df2], dim = cc_dim)
    
    return df1

In [5]:
def open_database_abs_file_xr(model, var, year):
    
    '''This function opens and returns an xarray dataset for a given:
       model : one of the six available model outputs in the database (does not work for UFS)
       var   : any var present in the database (must match an available var or will throw an error)
       year  : same as var, but for year instead'''

    path = f'/raid60B/s2sfire/NOAA_S2S/database_files/{model}/'
    base = get_filename(model)
    name = f'{path}{var}_{base}_Abs_{str(year)}.nc'

    df = xr.open_dataset(name, decode_times=True)[var]
    return df

### Harmonics Function

In [6]:
def compute_harmonics(yt, k=5, offbyone=False):
    
    '''This function is based off of Section 8.4 (Frequency Domain - 1. Harmonic Analysis)
        from the textbook Statistical Methods in the Atmospheric Sciences by Wilks (2006).
        This function calculates a specified number of harmonics, or fitted cosine curves,
        for a given time series. It functions very similiarly to fourier analysis.
        
        Inputs: 
        yt : a 1-d time series to perform harmonics analysis on
        k  : the number of harmonics to compute
        
        Outputs: 
        yt0  : the sum of the k-harmonics computed
        ybar : the mean of yt
        C    : the coefficient for each harmonic
        w    : the frequency adjustment for each harmonic
        
        KEYWORDS
        offbyone : use if your iterator should start at 1 (Python iterators start at 0)
                   (do not use if unsure; for long time series (100+), it won't matter)'''

    n = yt.shape[-1]                                                                                                          # Make time the last dimension 
                       
    if (k > int(n/2)):                                                                                                        # Ensure the number of harmonics is within the valid range
        k = int(n/2)                                                                                                          # Set k to the maximum valid number of harmonics
                       
    x    = np.linspace(1, n, n) - 1                                                                                           # Create an array of time indices
    ybar = np.nanmean(yt, axis=-1, keepdims=True)                                                                             # Compute the mean of yt along the time dimension
    freq = 2 * np.pi * x / n                                                                                                  # Compute the frequency array
                   
    harmonics_range = np.arange(1, k + 1)                                                                                     # Create an array of harmonic indices
                   
    cos_terms = np.cos(freq * harmonics_range[:, np.newaxis])[np.newaxis, np.newaxis, ...]                                    # Compute cosine terms for harmonics
    sin_terms = np.sin(freq * harmonics_range[:, np.newaxis])[np.newaxis, np.newaxis, ...]                                    # Compute sine terms for harmonics
                       
    A = (2 / n) * np.nansum(yt[:, :, np.newaxis, :] * cos_terms, axis=-1)                                                     # Compute cosine coefficients
    B = (2 / n) * np.nansum(yt[:, :, np.newaxis, :] * sin_terms, axis=-1)                                                     # Compute sine coefficients
                   
    C = np.sqrt(A**2 + B**2)                                                                                                  # Compute the amplitude of the harmonics
                   
    w = np.where(A == 0, np.pi / 2, np.arctan(B / A))                                                                         # Compute the phase angle
    w = np.where(A < 0, w + np.pi, w)                                                                                         # Adjust phase angle for negative cosine coefficients
    w = np.where(w >= 2 * np.pi, w - 2 * np.pi, w)                                                                            # Ensure phase angle is within the range [0, 2π]
                   
    if (offbyone):                                                                                                            # Check for off-by-one error
        w += np.deg2rad(1 / n)                                                                                                # Adjust phase angle to correct off-by-one error
                   
    yt0 = np.repeat(ybar, n, axis=-1)                                                                                         # Initialize the reconstructed time series with the mean
                   
    freq_adjusted = freq[np.newaxis, np.newaxis, :, np.newaxis]                                                               # Adjust frequency array for broadcasting
                   
    harmonics_range = harmonics_range[np.newaxis, np.newaxis, np.newaxis, :]                                                  # Adjust harmonic range array for broadcasting
                   
    yt0 = yt0 + np.sum(C[:, :, np.newaxis, :] * np.cos(harmonics_range * freq_adjusted - w[:, :, np.newaxis, :]), axis=-1)    # Reconstruct the time series using harmonics

    return (yt0, ybar.squeeze(axis=-1), C, w)                                                                                 # Return the reconstructed time series, mean, amplitude, and phase angle

## Application of the Harmonics Function

Organized by model as each needs handled differently.

### ERA5 (Initial testing)

In [ ]:
#these cells applies the harmonics to the full period available for each variable for ERA5 data
#use this one! this is the more useful application of harmonics

k = 5

filepath = '../database_files/ERA5/'
filebase = '_ERA5_REANALYSIS_Abs_'
suffix   = '.nc'

var_names = [
   'blh' ,   'cape',    'cp',     'd2m',   'ffwi',
   'hdwi',   'i10fg',   'lsrr',   'rh',    'swvl1',
   't2m',    'tp',      'u10',    'v10',   'vpd',
   'wdir',   'wspeed']

In [ ]:
for var_name in var_names:                                                                                             # Loop through each variable name in var_names
    raw_df = xr.open_mfdataset(f'{filepath}{var_name}{filebase}*{suffix}', decode_times=True)[var_name].load()         # Open and load the dataset for the current variable
      
    print(f'Applying Harmonics to: {var_name}')                                                                        # Print a message indicating the current variable being processed
                
    ufunc_output = xr.apply_ufunc(                                                                                     # Apply the compute_harmonics function using xarray's apply_ufunc
        compute_harmonics,                                                                                             # Function to apply
        raw_df,                                                                                                        # Input dataset
        k,                                                                                                             # Number of harmonics
        False,                                                                                                         # Off-by-one correction flag
        input_core_dims=[['time'], [], []],                                                                            # Input core dimensions
        output_core_dims=[['time'], [], ['k'], ['k']],                                                                 # Output core dimensions
        output_dtypes=[float, float, float, float],                                                                    # Output data types
        vectorize=False)                                                                                               # Do not vectorize the function
                
    climo = ufunc_output[0].transpose('time', 'latitude', 'longitude').rename(var_name + '_climo')                     # Extract and rename the climatology component
    anoms = raw_df - climo                                                                                             # Compute anomalies by subtracting climatology from raw data
    anoms = anoms.rename(var_name + '_anoms')                                                                          # Rename the anomalies component
                
    means  = ufunc_output[1].rename(var_name + '_means')                                                               # Extract and rename the means component
    consts = ufunc_output[2].rename(var_name + '_consts')                                                              # Extract and rename the constants component
    phis   = ufunc_output[3].rename(var_name + '_phis')                                                                # Extract and rename the phase angles component
                
    climo_data = xr.merge([climo, means, consts, phis])                                                                # Merge climatology components into a single dataset
                
    print(f'Complete. Writing anomaly and climatology files over full period for {var_name}...')                       # Print a message indicating completion of processing
      
    anoms.to_netcdf(os.path.join(filepath + '/Anoms/', var_name + filebase + 'anoms_full_period' + suffix))            # Save anomalies to a NetCDF file
    climo_data.to_netcdf(os.path.join(filepath + '/Climos/', var_name + filebase + 'climos_full_period' + suffix))     # Save climatology data to a NetCDF file


### NCEP

In [ ]:
#these cells applies the harmonics to the full period available for each variable for NCEP data
#use this one! this is the more useful application of harmonics

k = 5

filepath = '../database_files/NCEP/'
filebase = '_NCEP_RENALYSIS_II_Abs_'
suffix   = '.nc'

var_names = [
    'air',   'ffwi', 'hdwi', 'prate', 'rhum', 
    'soilw', 'uwnd', 'vpd',  'vwnd',  'wdir', 
    'wspeed']

In [ ]:
for var_name in var_names:                                                                                                  # Loop through each variable name in var_names
    raw_df = xr.open_mfdataset(f'{filepath}{var_name}{filebase}*{suffix}', decode_times=True)[var_name].load()              # Open and load the dataset for the current variable
    raw_df = raw_df.sortby('time').squeeze()                                                                                # Sort the dataset by time and remove any singleton dimensions
    
    print(f'Applying Harmonics to: {var_name}')                                                                             # Print a message indicating the current variable being processed
                     
    ufunc_output = xr.apply_ufunc(                                                                                          # Apply the compute_harmonics function using xarray's apply_ufunc
        compute_harmonics,                                                                                                  # Function to apply
        raw_df,                                                                                                             # Input dataset
        k,                                                                                                                  # Number of harmonics
        False,                                                                                                              # Off-by-one correction flag
        input_core_dims=[['time'], [], []],                                                                                 # Input core dimensions
        output_core_dims=[['time'], [], ['k'], ['k']],                                                                      # Output core dimensions
        output_dtypes=[float, float, float, float],                                                                         # Output data types
        vectorize=False)                                                                                                    # Do not vectorize the function
                     
    climo = ufunc_output[0].transpose('time', 'lat', 'lon').rename(var_name + '_climo')                                     # Extract and rename the climatology component
    anoms = raw_df - climo                                                                                                  # Compute anomalies by subtracting climatology from raw data
    anoms = anoms.rename(var_name + '_anoms')                                                                               # Rename the anomalies component
                     
    means  = ufunc_output[1].rename(var_name + '_means')                                                                    # Extract and rename the means component
    consts = ufunc_output[2].rename(var_name + '_consts')                                                                   # Extract and rename the constants component
    phis   = ufunc_output[3].rename(var_name + '_phis')                                                                     # Extract and rename the phase angles component
                     
    climo_data = xr.merge([climo, means, consts, phis])                                                                     # Merge climatology components into a single dataset
                     
    print(f'Complete. Writing anomaly and climatology files over full period for {var_name}...')                            # Print a message indicating completion of processing
    
    anoms.to_netcdf(os.path.join(filepath + '/Anoms/', var_name + filebase + 'anoms_full_period' + suffix))                 # Save anomalies to a NetCDF file
    climo_data.to_netcdf(os.path.join(filepath + '/Climos/', var_name + filebase + 'climos_full_period' + suffix))          # Save climatology data to a NetCDF file



### NARR

In [ ]:
narr_years     = np.linspace(2011, 2014, 4).astype(int).astype(str)
narr_filebase  = '_NARR_REANALYSIS_Abs_'
narr_filepath = '../database_files/NARR/'
narr_var_names = [
    'ffwi', 'hdwi', 
    'Planetary_boundary_layer_height_surface', 
    'Precipitation_rate_surface', 
    'Relative_humidity_height_above_ground', 
    'Soil_moisture_content_layer_between_two_depths_below_surface_layer', 
    'Temperature_height_above_ground', 
    'Total_precipitation_surface_3_Hour_Accumulation', 
    'u-component_of_wind_height_above_ground', 
    'v-component_of_wind_height_above_ground', 
    'vpd', 'wdir', 'wspeed']

In [ ]:
k = 3                                                                                                                  # Number of harmonics to compute
            
for var in narr_var_names:                                                                                             # Loop through each variable name in narr_var_names
    for year, i in zip(narr_years, np.arange(len(narr_years))):                                                        # Loop through each year and its index
        narr_buffer = open_database_abs_file_xr('NARR', var, year)                                                     # Open the dataset for the current variable and year
        if ('height_above_ground2' in narr_buffer.dims):                                                               # Check if 'height_above_ground2' dimension exists
            narr_buffer = narr_buffer.sel(height_above_ground2=10.0)                                                   # Select data at 10 meters above ground
        if ('height_above_ground' in narr_buffer.dims):                                                                # Check if 'height_above_ground' dimension exists
            narr_buffer = narr_buffer.sel(height_above_ground=2.0)                                                     # Select data at 2 meters above ground
        if ('reftime' in narr_buffer.coords):                                                                          # Check if 'reftime' coordinate exists
            narr_buffer = narr_buffer.drop('reftime')                                                                  # Drop the 'reftime' coordinate
        narr_buffer = narr_buffer.squeeze().astype('float32')                                                          # Squeeze and convert data to float32
                    
        if (i == 0):                                                                                                   # If it's the first year
            narr_df = narr_buffer.copy()                                                                               # Initialize narr_df with the first year's data
        else:                                                                                                          # For subsequent years
            narr_df = xr.concat([narr_df, narr_buffer], dim='time')                                                    # Concatenate data along the time dimension
                
    print(f'Reading files complete. Final dataframe shape: {narr_df.shape}')                                           # Print the shape of the final dataframe
            
    print(f'Applying Harmonics to: {var}')                                                                             # Print a message indicating the current variable being processed
                
    ufunc_output = xr.apply_ufunc(                                                                                     # Apply the compute_harmonics function using xarray's apply_ufunc
        compute_harmonics,                                                                                             # Function to apply
        narr_df,                                                                                                       # Input dataset
        k,                                                                                                             # Number of harmonics
        False,                                                                                                         # Off-by-one correction flag
        input_core_dims=[['time'], [], []],                                                                            # Input core dimensions
        output_core_dims=[['time'], [], ['k'], ['k']],                                                                 # Output core dimensions
        output_dtypes=[float, float, float, float],                                                                    # Output data types
        vectorize=False)                                                                                               # Do not vectorize the function
                
    climo = ufunc_output[0].transpose('time', 'y', 'x').rename(f'{var}_climo')                                         # Extract and rename the climatology component
    anoms = narr_df - climo                                                                                            # Compute anomalies by subtracting climatology from raw data
    anoms = anoms.rename(f'{var}_anoms')                                                                               # Rename the anomalies component
                
    means  = ufunc_output[1].rename(f'{var}_means')                                                                    # Extract and rename the means component
    consts = ufunc_output[2].rename(f'{var}_consts')                                                                   # Extract and rename the constants component
    phis   = ufunc_output[3].rename(f'{var}_phis')                                                                     # Extract and rename the phase angles component
                
    climo_data = xr.merge([climo, means, consts, phis])                                                                # Merge climatology components into a single dataset
                
    print(f'Complete. Writing anomaly and climatology files over full period for {var}...')                            # Print a message indicating completion of processing
                
    anoms.to_netcdf(os.path.join(narr_filepath + '/Anoms/', f'{var}{narr_filebase}anoms_full_period.nc'))              # Save anomalies to a NetCDF file
    climo_data.to_netcdf(os.path.join(narr_filepath + '/Climos/', f'{var}{narr_filebase}climos_full_period.nc'))       # Save climatology data to a NetCDF file



### CONUS404

In [ ]:
#these cells applies the harmonics to the full period available for each variable for CONUS404 data
k = 5

filepath = '../database_files/CONUS404/SplitFiles/TwiceSplit/'
filebase = '_CONUS404_ANALYSIS_Abs_'
suffix   = '.nc'

var_names = [
    'ffwi', 'hdwi',   'MLCAPE', 'PBLH', 'PREC_ACC_NC',
    'rh',   'SBCAPE', 'SMOIS',  'T2',   'TD2',
    'U10',  'V10',    'vpd',    'wdir', 'wspeed']

In [ ]:
for var_name in var_names:                                                                                                                                    # Loop through each variable name in var_names
    for q1 in (np.arange(4) + 1).astype('str'):                                                                                                               # Loop through the first quadrant index (1 to 4)
        for q2 in (np.arange(4) + 1).astype('str'):                                                                                                           # Loop through the second quadrant index (1 to 4)
            raw_df = xr.open_mfdataset(f'{filepath}{var_name}{filebase}*_quad{q1}{q2}{suffix}', decode_times=True)[var_name].load().astype('float32')         # Open and load the dataset for the current variable and quadrant
                    
            print(f'Applying Harmonics to: CONUS404 {var_name} quad{q1}{q2}')                                                                                 # Print a message indicating the current variable and quadrant being processed
                                                                
            ufunc_output = xr.apply_ufunc(                                                                                                                    # Apply the compute_harmonics function using xarray's apply_ufunc
                compute_harmonics,                                                                                                                            # Function to apply
                raw_df,                                                                                                                                       # Input dataset
                k,                                                                                                                                            # Number of harmonics
                False,                                                                                                                                        # Off-by-one correction flag
                input_core_dims=[['Time'], [], []],                                                                                                           # Input core dimensions
                output_core_dims=[['Time'], [], ['k'], ['k']],                                                                                                # Output core dimensions
                output_dtypes=[float, float, float, float],                                                                                                   # Output data types
                vectorize=False)                                                                                                                              # Do not vectorize the function
                    
            climo = ufunc_output[0].transpose('Time', 'south_north', 'west_east').rename(var_name + '_climo')                                                 # Extract and rename the climatology component
            anoms = raw_df - climo                                                                                                                            # Compute anomalies by subtracting climatology from raw data
            anoms = anoms.rename(var_name + '_anoms')                                                                                                         # Rename the anomalies component
                                                               
            means  = ufunc_output[1].rename(var_name + '_means')                                                                                              # Extract and rename the means component
            consts = ufunc_output[2].rename(var_name + '_consts')                                                                                             # Extract and rename the constants component
            phis   = ufunc_output[3].rename(var_name + '_phis')                                                                                               # Extract and rename the phase angles component
                                                               
            climo_data = xr.merge([climo, means, consts, phis])                                                                                               # Merge climatology components into a single dataset
                                                               
            print(f'Complete. Writing anomaly and climatology files over full period for {var_name}...')                                                      # Print a message indicating completion of processing
            
            anoms.to_netcdf(f'{filepath}/Anoms/{var_name}{filebase}anoms_full_period_quad{q1}{q2}{suffix}')                                                   # Save anomalies to a NetCDF file
            climo_data.to_netcdf(f'{filepath}/Climos/{var_name}{filebase}climos_full_period_quad{q1}{q2}{suffix}')                                            # Save climatology data to a NetCDF file


### HRRR

In [ ]:
k = 5

filepath = '../database_files/HRRR/SplitFiles/TwiceSplit/'

hrrr_years     = np.linspace(2014, 2018, 5).astype(int).astype(str)
hrrr_filebase  = '_HRRR_HISTORICAL_Abs_'
suffix         = '.nc'
hrrr_var_names = [
    'blh', 'cape', 'd2m', 'ffwi', 'gust', 
    'hdwi', 'mstav', 'prate', 'rh', 't2m', 
    'tp', 'u10', 'v10', 'vpd', 'wdir', 
    'wspeed']

In [ ]:
for var_name in hrrr_var_names:                                                                                                      # Loop through each variable name in hrrr_var_names
    for q1 in (np.arange(4) + 1).astype('str'):                                                                                      # Loop through the first quadrant index (1 to 4)
        for q2 in (np.arange(4) + 1).astype('str'):                                                                                  # Loop through the second quadrant index (1 to 4)
            raw_df = xr.open_mfdataset(f'{filepath}{var_name}{hrrr_filebase}*_quad{q1}{q2}{suffix}',                                 # Open and load the dataset for the current variable and quadrant
                                       decode_times=True)[var_name].load().astype('float32')                          
                                      
            print(f'Applying Harmonics to: HRRR {var_name} quad{q1}{q2}')                                                            # Print a message indicating the current variable and quadrant being processed
                                      
            ufunc_output = xr.apply_ufunc(                                                                                           # Apply the compute_harmonics function using xarray's apply_ufunc
                compute_harmonics,                                                                                                   # Function to apply
                raw_df,                                                                                                              # Input dataset
                k,                                                                                                                   # Number of harmonics
                False,                                                                                                               # Off-by-one correction flag
                input_core_dims=[['time'], [], []],                                                                                  # Input core dimensions
                output_core_dims=[['time'], [], ['k'], ['k']],                                                                       # Output core dimensions
                output_dtypes=[float, float, float, float],                                                                          # Output data types
                vectorize=False)                                                                                                     # Do not vectorize the function
                                      
            climo = ufunc_output[0].transpose('time', 'y', 'x').rename(var_name + '_climo')                                          # Extract and rename the climatology component
            anoms = raw_df - climo                                                                                                   # Compute anomalies by subtracting climatology from raw data
            anoms = anoms.rename(var_name + '_anoms')                                                                                # Rename the anomalies component
                                      
            means  = ufunc_output[1].rename(var_name + '_means')                                                                     # Extract and rename the means component
            consts = ufunc_output[2].rename(var_name + '_consts')                                                                    # Extract and rename the constants component
            phis   = ufunc_output[3].rename(var_name + '_phis')                                                                      # Extract and rename the phase angles component
                                      
            climo_data = xr.merge([climo, means, consts, phis])                                                                      # Merge climatology components into a single dataset
                                      
            print(f'Complete. Writing anomaly and climatology files over full period for {var_name}...')                             # Print a message indicating completion of processing
    
            anoms.to_netcdf(f'{filepath}/Anoms/{var_name}{hrrr_filebase}anoms_full_period_quad{q1}{q2}{suffix}')                     # Save anomalies to a NetCDF file
            climo_data.to_netcdf(f'{filepath}/Climos/{var_name}{hrrr_filebase}climos_full_period_quad{q1}{q2}{suffix}')              # Save climatology data to a NetCDF file


### NAM

In [ ]:
nam_years     = np.linspace(2011, 2018, 8).astype(int).astype(str)
nam_filebase  = '_NAM_HISTORICAL_Abs_'
nam_filepath  = '../database_files/NAM/'
nam_var_names = [
    'cape', 'ffwi', 'gust', 'hdwi', 'r',
    'sm',   't2m',  'tp',   'u10',  'v10',
    'vpd',  'wdir', 'wspeed']

In [ ]:
k = 5                                                                                                                    # Number of harmonics to compute
for var in nam_var_names:                                                                                                # Loop through each variable name in nam_var_names
    if var in ['cape', 'gust']:                                                                                          # Check if the variable is 'cape' or 'gust'
        years = np.linspace(2017, 2018, 2).astype(int).astype(str)                                                       # Set years to 2017 and 2018
        k = 1                                                                                                            # Set number of harmonics to 1
    elif var in ['ffwi', 'tp']:                                                                                          # Check if the variable is 'ffwi' or 'tp'
        years = np.linspace(2013, 2018, 6).astype(int).astype(str)                                                       # Set years from 2013 to 2018
    else:                                                                                                                # For other variables
        years = nam_years                                                                                                # Use the predefined nam_years
                      
    for year, i in zip(years, np.arange(len(years))):                                                                    # Loop through each year and its index
        nam_buffer = open_database_abs_file_xr('NAM', var, year)                                                         # Open the dataset for the current variable and year
        print(f'File read: NAM {var} for {year}')                                                                        # Print a message indicating the file read
                      
        if year == '2012':                                                                                               # Check if the year is 2012
            if var == 'hdwi':                                                                                            # Check if the variable is 'hdwi'
                drop_coords = ['step', 'valid_time']                                                                     # Set coordinates to drop
            elif var == 'sm':                                                                                            # Check if the variable is 'sm'
                drop_coords = ['step', 'depthBelowLandLayer', 'valid_time']                                              # Set coordinates to drop
            else:                                                                                                        # For other variables
                drop_coords = ['step', 'heightAboveGround', 'valid_time']                                                # Set coordinates to drop
            nam_buffer = nam_buffer.drop(drop_coords)                                                                    # Drop the specified coordinates
              
        if (var == 'sm' and year in ['2017', '2018']):                                                                   # Check if the variable is 'sm' and the year is 2017 or 2018
            nam_buffer = nam_buffer.sel(depthBelowLandLayer=0)                                                           # Select data at the surface layer
        nam_buffer = nam_buffer.sortby('time')                                                                           # Sort the dataset by time
        nam_buffer = nam_buffer.squeeze().astype('float32')                                                              # Squeeze and convert data to float32
                  
        if i == 0:                                                                                                       # If it's the first year
            nam_df = nam_buffer.copy()                                                                                   # Initialize nam_df with the first year's data
        else:                                                                                                            # For subsequent years
            nam_df = xr.concat([nam_df, nam_buffer], dim='time')                                                         # Concatenate data along the time dimension
                          
    print(f'Reading files complete. Final dataframe shape: {nam_df.shape}')                                              # Print the shape of the final dataframe
              
    print(f'Applying Harmonics to: {var}')                                                                               # Print a message indicating the current variable being processed
                  
    ufunc_output = xr.apply_ufunc(                                                                                       # Apply the compute_harmonics function using xarray's apply_ufunc
        compute_harmonics,                                                                                               # Function to apply
        nam_df,                                                                                                          # Input dataset
        k,                                                                                                               # Number of harmonics
        False,                                                                                                           # Off-by-one correction flag
        input_core_dims=[['time'], [], []],                                                                              # Input core dimensions
        output_core_dims=[['time'], [], ['k'], ['k']],                                                                   # Output core dimensions
        output_dtypes=[float, float, float, float],                                                                      # Output data types
        vectorize=False)                                                                                                 # Do not vectorize the function
                  
    climo = ufunc_output[0].transpose('time', 'y', 'x').rename(f'{var}_climo')                                           # Extract and rename the climatology component
    anoms = nam_df - climo                                                                                               # Compute anomalies by subtracting climatology from raw data
    anoms = anoms.rename(f'{var}_anoms')                                                                                 # Rename the anomalies component
                  
    means  = ufunc_output[1].rename(f'{var}_means')                                                                      # Extract and rename the means component
    consts = ufunc_output[2].rename(f'{var}_consts')                                                                     # Extract and rename the constants component
    phis   = ufunc_output[3].rename(f'{var}_phis')                                                                       # Extract and rename the phase angles component
                  
    climo_data = xr.merge([climo, means, consts, phis])                                                                  # Merge climatology components into a single dataset
                  
    print(f'Complete. Writing anomaly and climatology files over full period for {var}...')                              # Print a message indicating completion of processing
                  
    anoms.to_netcdf(os.path.join(nam_filepath + '/Anoms/', f'{var}{nam_filebase}anoms_full_period.nc'))                  # Save anomalies to a NetCDF file
    climo_data.to_netcdf(os.path.join(nam_filepath + '/Climos/', f'{var}{nam_filebase}climos_full_period.nc'))           # Save climatology data to a NetCDF file
